In [32]:
import vk
import time
from Levenshtein import distance

In [ ]:
https://vk.com/dev/implicit_flow_user
    
    переадресовываем на свою страницу домашнюю просто

In [4]:
token_id_leo_wall = "328b4e2e3ac7378e82954f199f66bad644bd39c21cb419d8f02a9082f0288c27e59ec0e59eec557c486b4"

In [83]:
session = vk.AuthSession(access_token=token_id_leo_wall)
api = vk.API(session, v='5.95', lang='ru', timeout=10)

api.wall.deleteComment(owner_id = '-15787787',comment_id = '150952')


1

In [119]:
api.wall.restoreComment(owner_id = '-15787787',comment_id = '148457')

1

In [112]:
api.users.get(user_ids = '24156876')


[{'id': 24156876,
  'first_name': 'Юлия',
  'last_name': 'Dandelion',
  'is_closed': True,
  'can_access_closed': False}]

In [99]:
api.account.getProfileInfo(owner_id = "2513435")

{'first_name': 'Lingua',
 'last_name': 'Leo',
 'sex': 2,
 'relation': 0,
 'bdate': '2.3.1960',
 'bdate_visibility': 1,
 'home_town': 'Москва',
 'country': {'id': 1, 'title': 'Россия'},
 'city': {'id': 1, 'title': 'Москва'},
 'status': '',
 'phone': '+7 *** *** ** 48'}

# BAD WORDS DETECTION

In [ ]:
from string import punctuation

In [66]:
from pymystem3 import Mystem
m = Mystem()
text = "Красивая мама красиво мыла раму"
lemmas = m.lemmatize(text)

In [50]:
bad_words_list = []
with open("bad_words.txt", "r") as f:
    for word in f.readlines():
        bad_words_list.append(word[:-1])
        if word[:-1].endswith("ть"):
            vozvr = word[:-1] + "ся"
            #print(vozvr)
            bad_words_list.append(vozvr) 
        #print("||",word[:-1],"||")
    

In [16]:
bad_words_list[0]

'авотхуй'

In [59]:
m.lemmatize("блядские")

['блядский', '\n']

In [28]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [117]:
distance("прет", "пердеть")

3

In [56]:
test_text = """Вероника, вас откровенно жалко, не в ту компанию вы устроились. 
Там заправляют бездумные недальновидные люди. Вот просто по человечески все понято, с вас спроса никакого. 
Сделать пост в социалочке, попытаться сгладить косяки коллег - жаль, но этого не достаточно, чтобы снять весь тот 
негатив, который накопился из-за наплевательского и непрофессионального подхода. 
Обещания и ожидания, вот что мы пользователи видим и будем видеть и дальше. 
Сам имею опыт работы в нескольких it-продуктовых компаниях, но чтобы так нафакапить в релизе и не исправить косяк 
за пару дней, такого нет, не помню ни разу. Даже если происходит что-то из разряда вон выходящего, накатывается бэкап, а разрабы
в режиме 24/7 пыхтят над исправлением косяка. Просто на ходу дырявый "титаник" НИКТО никогда не латает. 
Если любая из компаний в которых я работал/ю так, извините за мой "французкий",  бы, я бы первый уволился. 
Просто мне с профессиональной точки зрения с такими людьми не по пути. 
Терпения вам, и всего самого плохого вашим нелогичным коллегам..."""

In [81]:
#%%time
m = Mystem()

def detect_bad_words(line):
    needs_attention = False
    needs_attention_list = []
    clean_line = ''
    for char in line:
        if char not in punctuation:
            clean_line += char.lower()
        else:
            clean_line += ' '
    #print(clean_line)
    clean_line = clean_line.strip()
    clean_line_list = clean_line.split()
    for clean_word in clean_line_list:
        if clean_word in bad_words_list:
            return "direct_bad_word_equality", (clean_word,clean_word)
        clean_lemma = m.lemmatize(clean_word)[0]
        
        if clean_lemma in bad_words_list:
            return "lemma_bad_word_equality", (clean_word,clean_lemma)
        
        not_handle_levenstein = ['чмо','гад']
        for bad_word in bad_words_list:
            if bad_word not in not_handle_levenstein:
                #print(bad_word, clean_lemma)
                dst = distance(bad_word, clean_lemma)
                if dst == 1:
                    needs_attention = True
                    needs_attention_list.append((clean_word,bad_word))
    if needs_attention:
        #print (needs_attention_list)
        return "NEEDS_ATTETNION", needs_attention_list
    return "line is clear", None
res, word_pair = detect_bad_words(test_text)       
print(res,word_pair)

line is clear None


# iterate

In [115]:
def handle_comment_check_results(check_results, detected_items, delete_list, attention_list,detected_comment_id, wall_owner_id, comment_text,comment_author_id, debug = False):
    bad_found_states = ['direct_bad_word_equality','lemma_bad_word_equality']
    if debug == True and check_results in bad_found_states: print("BAD_FOUND", detected_items)
    if check_results in bad_found_states:
        author_info = api.users.get(user_ids = comment_author_id)
        api.wall.deleteComment(owner_id = wall_owner_id,comment_id = detected_comment_id)
        delete_json = {'comment_id':detected_comment_id, 'wall_owner_id':wall_owner_id, 'detccted_items':detected_items,"author":author_info,'text':comment_text}
        delete_list.append(delete_json)
    if debug == True and check_results == "NEEDS_ATTETNION": print("NEEDS_ATTETNION", detected_items)
    elif check_results == "NEEDS_ATTETNION":
        author_info = api.users.get(user_ids = comment_author_id)
        attention_json = {'comment_id':detected_comment_id, 'wall_owner_id':wall_owner_id, 'attention_items':detected_items,"author":author_info,'text':comment_text}
        attention_list.append(attention_json)

In [105]:
import datetime

In [106]:
datetime.datetime.now()

datetime.datetime(2019, 6, 20, 14, 19, 28, 449433)

In [107]:
print(datetime.datetime.now())

2019-06-20 14:19:38.642001


In [114]:
m = Mystem()
def iterate_within_wall(wall_owner_id, print_comments = False, print_detections = False):
    items_for_deletion = []
    items_for_attention = []
    
    wall = api.wall.get(owner_id = wall_owner_id, count = 50)
    posts = wall['items']
    for post in posts:
        if print_comments: print(post['text'])
        post_comments = api.wall.getComments(owner_id = wall_owner_id ,post_id = post['id'],sort = "asc")
        post_comments_count = post_comments['count']
        post_first_comment_id = post_comments['items'][0]['id']
        #comments = api.wall.getComments(owner_id = wall_owner_id ,post_id = post['id'],sort = "asc",count = 100)
        if print_comments:print("post_comments_count",post_comments_count,'post_first_comment_id',post_first_comment_id)
        if print_comments:print("============COMMENTS===============")
        for st_comment_id in range(post_first_comment_id,post_first_comment_id + post_comments_count, 100 ):
            if print_comments:print("post_id",post['id'], "st_comment_id",st_comment_id )
            comments = api.wall.getComments(post_id = post['id'], start_comment_id = st_comment_id, owner_id = wall_owner_id ,sort = "asc",count = 100)
            #print(comments)
            for comment in comments['items']:
                if "deleted" not in comment:
                    #print(comment)
                    if print_comments:
                        print(comment['id'])
                        print(comment['text'])
                        print("\n")
                    check_results, detected_features = detect_bad_words(comment['text'])
                    print(api.users.get(user_ids = comment['from_id']))

                    handle_comment_check_results(check_results, detected_features, items_for_deletion, items_for_attention,comment['id'], wall_owner_id,comment['text'],comment['from_id'], debug = print_detections)
                    try:
                        comment_thread = api.wall.getComments(post_id = post['id'], comment_id = comment['id'], owner_id = wall_owner_id)
                        if len(comment_thread['items']) > 0:
                            if print_comments:print("********thread_begins********")
                            for thread_mes in comment_thread['items']:
                                if print_comments:
                                    print(thread_mes['id'])
                                    print(thread_mes['text'],'\n')
                                check_results, detected_features = detect_bad_words(thread_mes['text'])
                                handle_comment_check_results(check_results, detected_features, items_for_deletion, items_for_attention,thread_mes['id'], wall_owner_id,thread_mes['text'],thread_mes['from_id'], debug = print_comments)
                            if print_comments:
                                print("********thread_ends********\n")
                    except:
                        if print_comments: print("parent comment was deleted")
                        pass
                time.sleep(1)
        if print_comments:
            print("==============================")
        
        
        time.sleep(1)
        
        return items_for_deletion, items_for_attention
deleted, attention = iterate_within_wall("-15787787",print_comments = False, print_detections = True)

[{'id': 24156876, 'first_name': 'Юлия', 'last_name': 'Dandelion', 'is_closed': True, 'can_access_closed': False}]
[{'id': 2513435, 'first_name': 'Igor', 'last_name': 'Sushko', 'is_closed': True, 'can_access_closed': False}]
[{'id': 24722291, 'first_name': 'Владимир', 'last_name': 'Архипов', 'is_closed': False, 'can_access_closed': True}]
NEEDS_ATTETNION [('ура', 'дура')]
[{'id': 7747135, 'first_name': 'Вадим', 'last_name': 'Мелик-Касумов', 'is_closed': True, 'can_access_closed': False}]
[{'id': 5669548, 'first_name': 'Alina', 'last_name': 'Radeva', 'is_closed': False, 'can_access_closed': True}]
NEEDS_ATTETNION [('верните', 'пернуть')]
[{'id': 282484904, 'first_name': 'Polina', 'last_name': 'Sokolsky', 'is_closed': True, 'can_access_closed': False}]
NEEDS_ATTETNION [('для', 'бля'), ('для', '6ля'), ('для', 'бля')]


KeyboardInterrupt: 

In [116]:
deleted

[{'comment_id': 151160,
  'wall_owner_id': '-15787787',
  'detccted_items': ('гад', 'гад'),
  'author': [{'id': 3269521,
    'first_name': 'Николай',
    'last_name': 'Бабаков',
    'is_closed': False,
    'can_access_closed': True}]}]

In [91]:
attention

[{'comment_id': 151148,
  'wall_owner_id': '-15787787',
  'attention_items': [('для', 'бля'), ('для', '6ля'), ('для', 'бля')]},
 {'comment_id': 151072,
  'wall_owner_id': '-15787787',
  'attention_items': [('для', 'бля'), ('для', '6ля'), ('для', 'бля')]},
 {'comment_id': 151063,
  'wall_owner_id': '-15787787',
  'attention_items': [('для', 'бля'), ('для', '6ля'), ('для', 'бля')]},
 {'comment_id': 151042,
  'wall_owner_id': '-15787787',
  'attention_items': [('для', 'бля'), ('для', '6ля'), ('для', 'бля')]},
 {'comment_id': 151049,
  'wall_owner_id': '-15787787',
  'attention_items': [('врать', 'срать')]},
 {'comment_id': 151050,
  'wall_owner_id': '-15787787',
  'attention_items': [('вру', 'срать')]},
 {'comment_id': 151146,
  'wall_owner_id': '-15787787',
  'attention_items': [('души', 'дура')]},
 {'comment_id': 151124,
  'wall_owner_id': '-15787787',
  'attention_items': [('плюшки', 'шлюшка')]},
 {'comment_id': 151134,
  'wall_owner_id': '-15787787',
  'attention_items': [('врут', 'ср